<a href="https://colab.research.google.com/github/gundamp/Match_Declined_Occupation_to_Closest_Accepted_Occupation/blob/main/classify_declines_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Find closest match for declined occupations</h1>

## **Import libraries and load data**

In [ ]:
import time
import os
import random 
import re
import string
import seaborn as sns
import itertools


from collections import Counter
from collections import defaultdict

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import seaborn as sns
import matplotlib

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize #nltk tokeniser
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

import gensim #for word embedding/vectorisation
from gensim.models import Word2Vec

import pickle

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer


from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 5.3 MB 66.8 MB/s 
     |████████████████████████████████| 1.3 MB 52.0 MB/s 
     |████████████████████████████████| 163 kB 57.3 MB/s 
     |████████████████████████████████| 7.6 MB 55.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=053bd0033aea397aecc32815c19aae550481114ab84ad2121a2462dfc64ef69c
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
# Import raw data

#data_raw = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
#data_raw.head()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## These are the non-declined occupations - descriptions and codes

path = "/content/drive/MyDrive/2021_Analytics_work/NLP/train.csv"
data_raw = pd.read_csv(path, encoding = 'ISO-8859-1')
data_raw.head()

,train_occ,train_label
0,Accounting: Clerk/Bookkeeper - average income ...,AA
1,Accounting: Clerk/Bookkeeper - average income ...,A
2,"Actuary - not degree qualified, average income...",AAA
3,"Actuary - not degree qualified, average income...",AA
4,Actuary - degree qualified,AAA


In [ ]:
# These are the declined occupations which we'll try to match against the above

path = "/content/drive/MyDrive/2021_Analytics_work/NLP/test.csv"
data_decline_raw = pd.read_csv(path, encoding = 'ISO-8859-1')
data_decline_raw.head()

,Declines
0,Acrobat - not performing at heights
1,Acrobat - performing at heights with adequate ...
2,Actor
3,Acupuncturist - not a member of Australian Acu...
4,Aerial/Antennae Erector - over ten metres


## **NLP pipline**

In [ ]:
# Encode occupation codes - only need this step for the classification model, not cosine similarities

conditions_encode_response = [
    
    (data_raw['train_label'] == 'A'),
    (data_raw['train_label'] == 'AA'),
    (data_raw['train_label'] == 'AAA'),
    (data_raw['train_label'] == 'AL'),
    (data_raw['train_label'] == 'AM'),
    (data_raw['train_label'] == 'B'),
    (data_raw['train_label'] == 'B5'),
    (data_raw['train_label'] == 'BB'),
    (data_raw['train_label'] == 'BB5'),
    (data_raw['train_label'] == 'C'),
    (data_raw['train_label'] == 'C2'),
    (data_raw['train_label'] == 'C5'),
    (data_raw['train_label'] == 'CC'),
    (data_raw['train_label'] == 'CC5'),
    (data_raw['train_label'] == 'SR'),
    (data_raw['train_label'] == 'SR2'),
    (data_raw['train_label'] == 'SR5')

    
    ]

float_condition = [
                    3,
                    2,
                    0,
                    0,
                    1,
                    4,
                    4,
                    2,
                    3,
                    6,
                    6,
                    6,
                    5,
                    5,
                    5,
                    4,
                    4

                ]


data_raw['response'] = np.select(conditions_encode_response, float_condition)

## Text cleaning for the non-declined occupations

- Remove special characters, inspect what has been removed
- Lowercase
- Remove duplicate whitespaces
- Add back ">" or "<" with "greater than" or "less than", as they infer income which drives certain occupation classes

In [ ]:
text_raw = data_raw['train_occ']
text_raw.head()

0    Accounting: Clerk/Bookkeeper - average income ...
1    Accounting: Clerk/Bookkeeper - average income ...
2    Actuary - not degree qualified, average income...
3    Actuary - not degree qualified, average income...
4                           Actuary - degree qualified
Name: train_occ, dtype: object

In [ ]:
# Check that the regular expression for removing symbols, such as "-", ":" and etc. works

print(text_raw[0])
print('\n')
print(re.sub(r'[^\w\s]', '', str(text_raw[0]).strip()))


Accounting: Clerk/Bookkeeper - average income >$80,000


Accounting ClerkBookkeeper  average income 80000


In [ ]:
# Remove special characters

text_clean_1 = []

for texts in text_raw:
    text_clean_1.append((re.sub(r'[^\w\s]', '', str(texts).strip())))

#text_clean_1

In [ ]:
# Lowercase

text_clean_2 = []

for texts in text_clean_1:
    text_clean_2.append(texts.lower())

#text_clean_2

In [ ]:
# Remove duplicate whitespaces

text_clean_3 = []

for texts in text_clean_2:
    text_clean_3.append(" ".join(texts.split()))

#text_clean_3


#" ".join(text_clean_2[0].split())

In [ ]:
#re.findall(re.compile(r'[^\w\s]'), text_raw[0])

characters_removed = []

for texts in text_raw:
    characters_removed.append(re.findall(re.compile(r'[^\w\s]'), texts))

type(characters_removed)

list

In [ ]:
#characters_removed

In [ ]:
income_boolean_greater_than = []
income_boolean_less_than = []

for characters in characters_removed:
    income_boolean_greater_than.append(('>' in characters))
    income_boolean_less_than.append(('<' in characters))

In [ ]:
d_concat = {'characters':characters_removed,'boolean_>':income_boolean_greater_than, 'boolean_<':income_boolean_less_than}
#d_concat

In [ ]:
df_concat = pd.DataFrame(d_concat)
df_concat.head()

,characters,boolean_>,boolean_<
0,"[:, /, -, >, $, ,]",True,False
1,"[:, /, -, <, $, ,]",False,True
2,"[-, ,, >, $, ,]",True,False
3,"[-, ,, <, $, ,]",False,True
4,[-],False,False


In [ ]:
d_text_clean_3 = {'text_clean': text_clean_3}
text_clean_3_df = pd.DataFrame(d_text_clean_3)
text_clean_3_df.head()

,text_clean
0,accounting clerkbookkeeper average income 80000
1,accounting clerkbookkeeper average income 80000
2,actuary not degree qualified average income 12...
3,actuary not degree qualified average income 12...
4,actuary degree qualified


## Text cleaning for the declined occupations

- Remove special characters, inspect what has been removed
- Lowercase
- Remove duplicate whitespaces
- Add back ">" or "<" with "greater than" or "less than", as they infer income which drives certain occupation classes

**The last step is not essential as the declined occupations data don't have these operators**


In [ ]:
text_decline_raw = data_decline_raw['Declines']
text_decline_raw.head()

0                  Acrobat - not performing at heights
1    Acrobat - performing at heights with adequate ...
2                                                Actor
3    Acupuncturist - not a member of Australian Acu...
4            Aerial/Antennae Erector - over ten metres
Name: Declines, dtype: object

In [ ]:
text_decline_clean_1 = []

for texts in text_decline_raw:
    text_decline_clean_1.append((re.sub(r'[^\w\s]', '', str(texts).strip())))

#text_decline_clean_1

In [ ]:
# Check characters removed, there doesn't seem to be ">" or "<"
characters_decline_removed = []

for texts in text_decline_raw:
    characters_decline_removed.append(re.findall(re.compile(r'[^\w\s]'), texts))

#characters_decline_removed

In [ ]:
# Lowercase

text_decline_clean_2 = []

for texts in text_decline_clean_1:
    text_decline_clean_2.append(texts.lower())

#text_decline_clean_2

In [ ]:
# Remove duplicate whitespaces

text_decline_clean_3 = []

for texts in text_decline_clean_2:
    text_decline_clean_3.append(" ".join(texts.split()))

#text_decline_clean_3

In [ ]:
d_text_decline_clean_3 = {'text_decline_clean': text_decline_clean_3}
text_decline = pd.DataFrame(d_text_decline_clean_3)

## **BERT embedding for declined occupations**

In [ ]:
# The output we want is embeddings_decline which is a numpy array, this is used as input to the texts matching later on

bert_input_decline = text_decline['text_decline_clean'].tolist()
model_decline = SentenceTransformer('paraphrase-mpnet-base-v2') 
embeddings_decline = model_decline.encode(bert_input_decline, show_progress_bar=True)
embedding_BERT_decline = np.array(embeddings_decline)

print('Shape of list containing reviews and their BERT weights:', embeddings_decline.shape)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Shape of list containing reviews and their BERT weights: (223, 768)


In [ ]:
df_embedding_bert_decline = pd.DataFrame(embeddings_decline)
#df_embedding_bert_decline.head()

## **BERT embeddings for non-declined occupations**

In [ ]:
# Concatenate dataframes, only used for classification model

data_model = pd.concat([text_clean_3_df, df_concat], axis = 1).drop('characters', axis = 1)
#print(data_model.head())
#print(data_model.info())


In [ ]:
# Replace greater than and less than operators with texts

conditions_cohort = [
    
    (data_model['boolean_>'] == 1),
    (data_model['boolean_<'] == 1),
    ((data_model['boolean_>'] == 0) & (data_model['boolean_<'] == 0))
    
    ]

strings_condition_cohort = ['greater', 'less', '']
data_model['income_boolean'] = np.select(conditions_cohort, strings_condition_cohort)

In [ ]:
data_model['text_clean_adj'] = data_model['text_clean'] + ' ' + data_model['income_boolean']
#data_model.info()

In [ ]:
# Inspect

pd.options.display.max_colwidth = 70
data_model['text_clean_adj']

0          accounting clerkbookkeeper average income 80000 greater
1             accounting clerkbookkeeper average income 80000 less
2       actuary not degree qualified average income 125000 greater
3          actuary not degree qualified average income 125000 less
4                                        actuary degree qualified 
                                   ...                            
1099                                                 wool classer 
1100                                    xray technician qualified 
1101                                      zoo attendant qualified 
1102                                    zoo attendant unqualified 
1103                                                    zoologist 
Name: text_clean_adj, Length: 1104, dtype: object

In [ ]:
data_model_bert = data_model.drop(['text_clean', 'boolean_>', 'boolean_<', 'income_boolean'], axis = 1)
data_model_bert.head()

,text_clean_adj
0,accounting clerkbookkeeper average income 80000 greater
1,accounting clerkbookkeeper average income 80000 less
2,actuary not degree qualified average income 125000 greater
3,actuary not degree qualified average income 125000 less
4,actuary degree qualified


In [ ]:
# Again the output we need is embeddings which is a numpy array, this is used as input to match texts later on

text = data_model_bert['text_clean_adj']
bert_input = text.tolist()
model = SentenceTransformer('paraphrase-mpnet-base-v2') 
embeddings = model.encode(bert_input, show_progress_bar=True)
embedding_BERT = np.array(embeddings)

print('Shape of list containing reviews and their BERT weights:', embeddings.shape)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Shape of list containing reviews and their BERT weights: (1104, 768)


In [ ]:
df_embedding_bert = pd.DataFrame(embeddings)
#df_embedding_bert.head()

In [ ]:
print(embedding_BERT.shape)
print(embedding_BERT_decline.shape)

(1104, 768)
(223, 768)


## **Use BERT embeddings to match a declined occupation to the closest non-declined occupation**

In [ ]:
len(embedding_BERT)

1104

In [ ]:
similarity_decline = cosine_similarity(embedding_BERT_decline, embedding_BERT)
similarity_decline.shape

(223, 1104)

In [ ]:
# Compile a dataframe which stores the similarity score ranging from 0 to 1 for each of the declined occupations (x 223) against each of the non-declined occupations (x 1104)

similarity_df = pd.DataFrame(similarity_decline)
similarity_df.shape

(223, 1104)

In [ ]:
# Return the position of the maximum similarity score for each row (i.e. each declined occupation)

index_similarity = similarity_df.idxmax(axis = 1)
index_similarity[0:5]

0     706
1     828
2    1035
3       5
4      14
dtype: int64

## **Check similarity outputs**

In [ ]:
# Use dataframe operations to return the English texts for the closest match for the declined occupations

data_inspect_interim = data_raw.iloc[index_similarity, :]
data_inspect_interim.shape

(223, 3)

In [ ]:
data_decline_raw.shape

(223, 1)

In [ ]:
data_inspect_interim.head()

,train_occ,train_label,response
706,"Painter - unqualified, more than three years experience, heights l...",C5,6
828,Scaffold Worker - up to ten metres,SR2,4
1035,Theatre/Cinema: Usher,CC,5
5,Acupuncturist - member of Australian Acupuncturist Association,AA,2
14,Aerial/Antennae Erector - up to ten metres,C2,6


In [ ]:
data_inspect = data_inspect_interim.reset_index(drop = True)

declined_occ = data_decline_raw['Declines']
non_declined_occ = data_inspect['train_occ']
non_declined_occ_code = data_inspect['train_label']
d_output = {'declined_occ': declined_occ, 'matched_desc': non_declined_occ, 'matched_desc_code': non_declined_occ_code}

In [ ]:
output_df = pd.DataFrame(d_output).reset_index(drop = True)

## **Inspect outputs**

In [ ]:
# Inspect a sample of 100

pd.set_option('max_rows', 100)
output_df.sample(n = 100, random_state = 44)

,declined_occ,matched_desc,matched_desc_code
73,Farrier/Blacksmith - less than three years experience,Farrier/Blacksmith - minimum three years experience,C2
108,Jockey: Flat racing,Saddlemaker,B
148,"Oil and Gas Industry: Rigger - onshore, height more than ten metres",Oil and Gas Industry: Driller - offshore,SR2
133,Model,Tyre Fitter/Repairer,C2
206,Student - medical/nursing/dentistry (with practical),Medical Practitioner,AM
45,Clothing Industry: Pattern Maker - working from home,Sewing Machine Mechanic,B
81,Forest Ranger - unqualified,Zoo Attendant - unqualified,SR2
44,Clothing Industry: Dressmaker - working from home,"Clothing Industry: Clothing/Fashion Designer - qualified, not work...",BB
156,"Personal Assistant - working from home, administration only","Personal Assistant - not working from home, administration only, a...",AA
51,Dancer,"Dancing Instructor - full time, not working from home",CC5


## Output to Excel/CSV



In [ ]:
# Extract files from the left pane (refresh first)

output_df.to_excel('output_occ.xlsx')

In [ ]:
# Export to HTML

%%shell
jupyter nbconvert --to html /content/classify_declines_BERT.ipynb

[NbConvertApp] Converting notebook /content/classify_declines_BERT.ipynb to html
[NbConvertApp] Writing 518886 bytes to /content/classify_declines_BERT.html


In [ ]:
text_decline['text_decline_clean']

0                                       acrobat not performing at heights
1          acrobat performing at heights with adequate safety precautions
2                                                                   actor
3      acupuncturist not a member of australian acupuncturist association
4                                  aerialantennae erector over ten metres
                                      ...                                
218                                      vehicle body builder unqualified
219                                                              wardsman
220                                              warehouse not management
221                    windscreen fitter less than three years experience
222                                                               wrecker
Name: text_decline_clean, Length: 223, dtype: object